##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Barren plateaus

(modified from the original)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/quantum/tutorials/barren_plateaus"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/quantum/blob/master/docs/tutorials/barren_plateaus.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/quantum/blob/master/docs/tutorials/barren_plateaus.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/quantum/docs/tutorials/barren_plateaus.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In this example you will explore the result of <a href="https://www.nature.com/articles/s41467-018-07090-4" class="external">McClean, 2019</a> that says not just any quantum neural network structure will do well when it comes to learning. In particular you will see that a certain large family of random quantum circuits do not serve as good quantum neural networks, because they have gradients that vanish almost everywhere. In this example you won't be training any models for a specific learning problem, but instead focusing on the simpler problem of understanding the behaviors of gradients.

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Update package resources to account for version changes.
import importlib, pkg_resources
importlib.reload(pkg_resources)

Now import TensorFlow and the module dependencies:

In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

np.random.seed(1234)
tf.random.set_seed(1234)

In [ ]:
from qcircuits.QCircuit import QCircuit
import json
from tqdm import tqdm
tf.get_logger().setLevel('ERROR')

In [ ]:
def process_batch(expectation, circuits, symbols, op):
    """Compute the variance of a batch of expectations w.r.t. op on each circuit that 
    contains `symbol`. Note that this method sets up a new compute graph every time it is
    called so it isn't as performant as possible."""
    
    # Setup a simple layer to batch compute the expectation gradients.
    expectation = tfq.layers.Expectation()

    # Prep the inputs as tensors
    circuit_tensor = tfq.convert_to_tensor(circuits)
    values_tensor = tf.convert_to_tensor(
        np.random.uniform(0, 2 * np.pi, (len(circuits), len(symbols))).astype(np.float32))

    # Use TensorFlow GradientTape to track gradients.
    with tf.GradientTape() as g:
        g.watch(values_tensor)
        forward = expectation(circuit_tensor,
                              operators=op,
                              symbol_names=symbols,
                              symbol_values=values_tensor)

    # Return variance of gradients across all circuits.
    grads = g.gradient(forward, values_tensor)
    grad_var = tf.math.reduce_std(grads, axis=0)
    return grad_var.numpy()[0]

### 3.1 Set up and run

Choose the number of random circuits to generate along with their depth and the amount of qubits they should act on. Then plot the results.

In [ ]:
# n_qubits = [2 * i for i in range(2, 7)
#            ]  # Ranges studied in paper are between 2 and 24.
n_qubits = [4, 8, 16]
# n_qubits = [4]
# n_layers = [1, 7, 15, 25]
n_layers = [1, 2]
print(list(n_layers))
# depth = 50  # Ranges studied in paper are between 50 and 500.
n_circuits = 200

with open('qcircuits/circuits_metadata.json') as json_file:
    circuit_metadata = json.load(json_file)

print(circuit_metadata['qc_pqc_dict'].keys())

# Setup a simple layer to batch compute the expectation gradients.
expectation = tfq.layers.Expectation()

thetas = {}
# for id in tqdm(circuit_metadata['qc_pqc_dict'].keys()):
for id in tqdm(['10_identity', '10']):
    theta_per_qubits = {}
    for n in n_qubits:
        theta_var = []
        for l in n_layers:
            try:
                qc = QCircuit(IEC_id='simple_encoding_y',
                        PQC_id=id,
                        MC_id='measure_all',
                        n_layers=l,
                        input_size=n,
                        p=0.01)
                circuit = qc.pqc_circuit()[0]
            except:
                continue
                
            circuits = [
                circuit for _ in range(n_circuits)
            ]
            op = qc.measurement_operators()

            symbols = []
            for i in range(int(qc.n_params)):
                symbols.append('theta{}'.format(i)) 

            theta_var.append(process_batch(expectation, circuits, symbols, op))
        theta_per_qubits[n] = theta_var
    
    thetas[id] = theta_per_qubits
print("Done")

In [ ]:
fig, axs = plt.subplots(1,len(n_qubits), sharex=True, sharey=True, figsize=(10,4))

plt.suptitle('Gradient Variance in QNNs')
plt.xticks(n_layers)

for i,n in enumerate(n_qubits):
    for id, theta_per_qubits in thetas.items():        
        if n in theta_per_qubits and len(theta_per_qubits[n]) > 0:
            axs[i].semilogy(n_layers, theta_per_qubits[n], label=f"{id}")
    axs[i].set_title(f"{n} qubits")
    axs[i].set_xlabel('n_layers')
    axs[i].set_ylabel('$\\partial \\theta$ variance')

handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right')
plt.show()

In [ ]:
# for id in tqdm(circuit_metadata['qc_pqc_dict'].keys()):
for id in ['qc10_pqc_identity']:
    qc = QCircuit(IEC_id='simple_encoding_y',
            PQC_id=id,
            MC_id='measure_all',
            n_layers=2,
            input_size=4,
            p=0.01)
    print(id)
    print("-------------------")
    circuit = qc.model_circuit_pqc_only()[0]
    print(circuit)
    print("-------------------")